In [ ]:
import pickle

import pandas as pd

In [ ]:
pororo_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/image_test/test/pororo_descriptions.csv', index_col=False)

## 파일별 디스크립션 1개씩 으로 조정, 파일경로+파일이름 합친 컬럼 생성

In [ ]:
for i in range(len(pororo_df['use_name'])):
    pororo_df['use_name'].iloc[i] = pororo_df['file_path'].iloc[i]+'/'+str(pororo_df['image_num'].iloc[i])

a = pororo_df.drop_duplicates(['file_path','image_num'])

a.reset_index(inplace=True)

a.drop('index', axis=1, inplace=True)

a.to_csv('/content/drive/MyDrive/Colab Notebooks/image_test/test/pororo_descriptions_2.csv', index=False) # 저장

In [ ]:
pororo_df_2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/image_test/test/pororo_descriptions_2.csv', index_col=False) # 로드
pororo_df_2

,file_path,image_num,description,use_name
0,Pororo_ENGLISH1_1_ep1,1,There is a sea over the mountains covered by s...,Pororo_ENGLISH1_1_ep1/1
1,Pororo_ENGLISH1_1_ep1,2,Pororo wears an aviator hat and goggles. Poror...,Pororo_ENGLISH1_1_ep1/2
2,Pororo_ENGLISH1_1_ep1,3,Pororo is saying hello with smile. There is a ...,Pororo_ENGLISH1_1_ep1/3
3,Pororo_ENGLISH1_1_ep1,4,Pororo found a round thing under the trees. Po...,Pororo_ENGLISH1_1_ep1/4
4,Pororo_ENGLISH1_1_ep1,5,Pororo sees a big egg with flecks of yellow un...,Pororo_ENGLISH1_1_ep1/5
...,...,...,...,...
16061,Pororo_The_Racing_Adventure_ep7,155,"Pipi Popo, two turtles, Tongtong, Rody, and Po...",Pororo_The_Racing_Adventure_ep7/155
16062,Pororo_The_Racing_Adventure_ep7,156,An old animal with a cane is in front of the c...,Pororo_The_Racing_Adventure_ep7/156
16063,Pororo_The_Racing_Adventure_ep7,157,"Loopy, Eddy, Crong, two turtles, Pororo, Petty...",Pororo_The_Racing_Adventure_ep7/157
16064,Pororo_The_Racing_Adventure_ep7,158,"Loopy, Eddy, Crong, two turtles, Pororo, Petty...",Pororo_The_Racing_Adventure_ep7/158


In [ ]:
pororo_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16066 entries, 0 to 16065
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   file_path    16066 non-null  object
 1   image_num    16066 non-null  int64 
 2   description  16066 non-null  object
 3   use_name     16066 non-null  object
dtypes: int64(1), object(3)
memory usage: 502.2+ KB


디스크립션 토크나이즈

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
all_captions = []
for cap in pororo_df_2['description']:
    cap = cap.replace("\ufffd\ufffd", " ")
    tokens = tokenizer.tokenize(cap.lower())
    tokens_new = []
    for t in tokens:
        t = t.encode('ascii', 'ignore').decode('ascii')
        if len(t) > 0:
            tokens_new.append(t)
    all_captions.append(tokens_new)
print(all_captions)

[['there', 'is', 'a', 'sea', 'over', 'the', 'mountains', 'covered', 'by', 'snow'], ['pororo', 'wears', 'an', 'aviator', 'hat', 'and', 'goggles', 'pororo', 'is', 'rolling', 'a', 'big', 'snowball', 'up', 'to', 'the', 'hilltop', 'the', 'hill', 'is', 'covered', 'by', 'snow', 'the', 'sky', 'is', 'blue', 'and', 'clear'], ['pororo', 'is', 'saying', 'hello', 'with', 'smile', 'there', 'is', 'a', 'big', 'snowball', 'on', 'the', 'hill', 'covered', 'by', 'snow', 'beside', 'pororo', 'the', 'sky', 'is', 'blue', 'and', 'clear'], ['pororo', 'found', 'a', 'round', 'thing', 'under', 'the', 'trees', 'pororo', 'feels', 'curious', 'pororo', 'starts', 'to', 'sweep', 'off', 'the', 'snow', 'around', 'it'], ['pororo', 'sees', 'a', 'big', 'egg', 'with', 'flecks', 'of', 'yellow', 'under', 'the', 'trees', 'pororo', 'gets', 'to', 'know', 'what', 'it', 'is', 'pororo', 'thinks', 'with', 'positive', 'mind'], ['pororo', 'gets', 'to', 'know', 'something', 'in', 'front', 'of', 'a', 'big', 'egg', 'with', 'flecks', 'of', 

딕셔너리 가져온다.
- text 임베딩 사전 크기 맞추기 위해

In [ ]:
from collections import defaultdict

In [ ]:
all_captions

In [ ]:
[captions_new, ixtoword, wordtoix, len(ixtoword)]

In [ ]:
with open('/content/captions.pickle', 'rb') as f:
    x = pickle.load(f)
ixtoword = x[2]
wordtoix = x[3]

In [ ]:
len(ixtoword)

27297

In [ ]:
captions_new=[]

In [ ]:
for t in all_captions:
    rev = []
    for w in t:
        if w in wordtoix:
            rev.append(wordtoix[w])
    # rev.append(0)  # do not need '<end>' token
    captions_new.append(rev)

In [ ]:
captions_new

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/image_test/test/captions.pickle', 'wb') as f: # 캡션 저장
    pickle.dump([captions_new, ixtoword, wordtoix], f, protocol=2)

with open('/content/drive/MyDrive/Colab Notebooks/image_test/test/filenames.pickle', 'wb') as f: # 파일명 저장
    pickle.dump(list(pororo_df_2['use_name']), f, protocol=2)

# 이미지 생성을 위한 사용자 입력값 캡션 변환


In [ ]:
from nltk.tokenize import RegexpTokenizer
import pickle
import torch
import pandas as pd


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/image_test/test/birds/birds/captions.pickle', 'rb') as f:
    x = pickle.load(f)
ixtoword = x[2]
wordtoix = x[3]

In [ ]:
cap = 'Pororo walking in the forest'

cap = cap.replace("\ufffd\ufffd", " ")
tokens = tokenizer.tokenize(cap.lower())
tokens_new = []
for t in tokens:
    t = t.encode('ascii', 'ignore').decode('ascii')
    if len(t) > 0:
        tokens_new.append(t)

print(tokens_new)

['pororo', 'walking', 'in', 'the', 'forest']


In [ ]:

def text2cap(s, word2idx):
    s = s.replace("\ufffd\ufffd", " ")
    tokens = tokenizer.tokenize(s.lower())
    tokens_new = []
    for t in tokens:
        t = t.encode('ascii', 'ignore').decode('ascii')
        if len(t) > 0:
            tokens_new.append(t)

    rev=[]
    for w in tokens_new:
        if w in word2idx:
            rev.append(word2idx[w])
    # rev.append(0)  # do not need '<end>' token
    while len(rev) < 18:
        rev.append(0)
    
    return rev, len(rev)

In [ ]:
cap = 'Pororo walking in the forest'
a, b = text2cap(cap, wordtoix)

In [ ]:
torch.tensor([a])

tensor([[4889,  267, 1010, 4399,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]])

In [ ]:
torch.tensor([b])

tensor([18])